<a href="https://colab.research.google.com/github/andylehti/Polyhedral-Index-Partition/blob/main/Polyhedral_Index_Partition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Old Optimized Polyhedral Index Partition

In [ ]:
import sys
from functools import lru_cache
from math import log10, ceil
import math
from mpmath import mp
import mpmath as mpm
sys.set_int_max_str_digits(0)
sys.setrecursionlimit(10000)

@lru_cache(maxsize=None)
def polyHedralIndex(x, y):
    if x < 1: return 0
    return int(mp.binomial(x + y - 1, y))

def approximate(n, t, x=1):
    e = polyHedralIndex(x, t)
    while n > e:
        x *= 10
        e = polyHedralIndex(x, t)
    if x == 10:
        while e > n:
            x -= 1
            e = polyHedralIndex(x, t)
        return x
    l, h = x // 10, x
    while l <= h:
        m = (l + h) // 2
        e = polyHedralIndex(m, t)
        if e == n: return m
        elif e < n: l = m + 1
        else: h = m - 1
    return l - 1 if polyHedralIndex(l - 1, t) < n else l

def getPair(k):
    k = mpm.mpf(k)
    s = mpm.floor(((-1 + (1 + 8 * k) ** 0.5) / 2))
    i = polyHedralIndex(s, 2)
    return int(s - (k - i)), int(k - i)

def getPartition(n, y):
    if y < 2: return [n]
    if y == 2: return getPair(n)
    a = approximate(n, y)
    p = polyHedralIndex(a, y)
    r = int(n - p)
    c = getPartition(r, y - 1)
    a = int(a - sum(c))
    return (a,) + c

def partition(n, y=2):
    mpm.mp.dps = 100
    mpm.mp.dps = 100 + len(str(n)) * 2
    return list(getPartition(n, y))

def pairInverse(a, b):
    a, b = mpm.mpf(a), mpm.mpf(b)
    n = a + b
    return int(mpm.nint(n * (n + 1) / 2 + b))

def getInverse(*a):
    a = list(a)
    s = len(''.join(map(str, a)))
    mpm.mp.dps = s * 2
    if len(a) <= 1: return a[-1]
    return inversePartition(*a)

def inversePartition(*c):
    if len(c) == 2: return pairInverse(*c)
    n = sum(c)
    r = inversePartition(*c[1:])
    return int(polyHedralIndex(n, len(c)) + r)

# New Optimized Polyhedral Index Partition

In [ ]:
from __future__ import annotations

from dataclasses import dataclass
from math import comb
from typing import Iterable, List, Sequence, Tuple

def polyHedralIndex(x: int, y: int) -> int:
    if y < 1:
        raise ValueError("dimension d must be >= 1")
    if x < 0:
        raise ValueError("magnitude m must be >= 0")
    return comb(x + y - 1, y)

def getInverse(n: Sequence[int], y: bool = False) -> int:
    if not n:
        raise ValueError("coords must be non-empty")
    a = list(n)
    if y:
        a = [zigzagEncode(z) for z in a]
    if any((not isinstance(v, int)) for v in a):
        raise TypeError("all coordinates must be integers")
    if any(v < 0 for v in a):
        raise ValueError("coordinates must be >= 0 (or use signed=True)")
    t = len(a)
    c = [0] * t
    s = 0
    for i in range(t - 1, -1, -1):
        s += a[i]
        c[i] = s
    k = 0
    for i in range(t):
        k += polyHedralIndex(c[i], t - i)
    return k

def partition(k: int, y: int, *, signed: bool = False) -> Tuple[int, ...]:
    if not isinstance(k, int) or k < 0:
        raise ValueError("index must be an integer >= 0")
    if not isinstance(y, int) or y < 1:
        raise ValueError("dim must be an integer >= 1")
    r = k
    p: List[int] = []
    for t in range(y, 0, -1):
        x = approximate(r, t)
        p.append(x)
        r -= polyHedralIndex(x, t)
    a: List[int] = []
    for i in range(y - 1):
        a.append(p[i] - p[i + 1])
    a.append(p[-1])
    if signed:
        a = [zigzagDecode(n) for n in a]
    return tuple(a)

def approximate(n: int, t: int) -> int:
    if n < 0:
        raise ValueError("remainder r must be >= 0")
    if n == 0:
        return 0
    l, h = 0, 1
    while polyHedralIndex(h, t) <= n:
        h *= 2
    l = h // 2
    while l + 1 < h:
        m = (l + h) // 2
        if polyHedralIndex(m, t) <= n:
            l = m
        else:
            h = m
    return l

def zigzagEncode(z: int) -> int:
    if not isinstance(z, int):
        raise TypeError("z must be int")
    return 2 * z if z >= 0 else -2 * z - 1

def zigzagDecode(n: int) -> int:
    if not isinstance(n, int) or n < 0:
        raise ValueError("n must be int >= 0")
    return n // 2 if (n % 2 == 0) else -(n // 2) - 1

# Playground
Ensure to press play on the version you wish to test out. They will overwrite each other.

In [ ]:
p = 320 # change for the number of partitions (dimensions)

n = 123456789 # change for the index (integer value to break into multidimensional coordinates)

a = partition(n, p)
r = getInverse(a)
print(a)
print(r)
print(r == n)

In [ ]:
def calculateIndices(n, y):
    p = partition(n, y)
    a = sum(p)
    e = len(p) - 1
    s = getInverse(([a] + [0] * e))
    c = getInverse(([0] * e + [a]))

    return {
        "integer_partitions": p,
        "cumulative_index": n,
        "integer_index": a,
        "set_index": e + 1,
        "partition_index_start": s,
        "partition_index_end": c,
        "partition_index": n - s
    }

result = calculateIndices(123456879, 3)
print(result)

In [ ]:
# increasing complexity test

n = 0
c = 1
f = 1
while True:
    a = partition(n, p)
    r = getInverse(a)
    print(n, a, r, n == r)
    c += 1
    f *= c
    n += f
    if c == 100: break

# Old vs. New Optimized Benchmark Test
1. Press play on the code below this text; and the code below the section: "New Optimized Polyhedral Index Partition."
2. Find the cell with: timerTest(a, b, c, d)
3. 'a' if 0 will benchmark only the new optimized code; if 1, then will benchmark against both old and new.
4. 'b' is the number of tests.
5. 'c' is the max number of dimensions to partition using: 2^c, so if c is 16, then the highest number of dimensions will be 65,536.
6. 'd' is the max integer value to break into dimensions (partitions) using the formula: 10^d, where if d is 10, then the max integer value will be: 10,000,000,000

In [ ]:
import sys
from functools import lru_cache
from math import log10, ceil
import math
from mpmath import mp
import mpmath as mpm
sys.set_int_max_str_digits(0)
sys.setrecursionlimit(100000)

@lru_cache(maxsize=None)
def old_polyHedralIndex(x, y):
    if x < 1: return 0
    return int(mp.binomial(x + y - 1, y))

def old_approximate(n, t, x=1):
    e = old_polyHedralIndex(x, t)
    while n > e:
        x *= 10
        e = old_polyHedralIndex(x, t)
    if x == 10:
        while e > n:
            x -= 1
            e = old_polyHedralIndex(x, t)
        return x
    l, h = x // 10, x
    while l <= h:
        m = (l + h) // 2
        e = old_polyHedralIndex(m, t)
        if e == n: return m
        elif e < n: l = m + 1
        else: h = m - 1
    return l - 1 if old_polyHedralIndex(l - 1, t) < n else l

def old_getPair(k):
    k = mpm.mpf(k)
    s = mpm.floor(((-1 + (1 + 8 * k) ** 0.5) / 2))
    i = old_polyHedralIndex(s, 2)
    return int(s - (k - i)), int(k - i)

def old_getPartition(n, y):
    if y < 2: return [n]
    if y == 2: return old_getPair(n)
    a = old_approximate(n, y)
    p = old_polyHedralIndex(a, y)
    r = int(n - p)
    c = old_getPartition(r, y - 1)
    a = int(a - sum(c))
    return (a,) + c

def old_partition(n, y=2):
    mpm.mp.dps = 100
    mpm.mp.dps = 100 + len(str(n)) * 2
    return list(old_getPartition(n, y))

def old_pairInverse(a, b):
    a, b = mpm.mpf(a), mpm.mpf(b)
    n = a + b
    return int(mpm.nint(n * (n + 1) / 2 + b))

def old_getInverse(*a):
    a = list(a)
    s = len(''.join(map(str, a)))
    mpm.mp.dps = s * 2
    if len(a) <= 1: return a[-1]
    return old_inversePartition(*a)

def old_inversePartition(*c):
    if len(c) == 2: return old_pairInverse(*c)
    n = sum(c)
    r = old_inversePartition(*c[1:])
    return int(old_polyHedralIndex(n, len(c)) + r)

import random
import time

def timerTest(e = 0, n = 100, r = 16, v = 100):
    tests = [
        (
            random.randint(10 ** (d - 1), 10 ** d - 1),
            random.randint(2, 2 ** r)
        )
        for d in (random.randint(1, v) for _ in range(n))
    ]

    newTotal = 0.0
    oldTotal = 0.0

    for i, (a, b) in enumerate(tests, 1):
        t0 = time.perf_counter()
        rNew = partition(a, b)
        t1 = time.perf_counter()
        dtNew = t1 - t0
        newTotal += dtNew

        if e == 1:
            t2 = time.perf_counter()
            rOld = old_partition(a, b)
            t3 = time.perf_counter()
            dtOld = t3 - t2
            oldTotal += dtOld
            same = tuple(rNew) == tuple(rOld)
            print(f"{i:02d} integer={a} dimensions={b} new={dtNew:.6f}s old={dtOld:.6f}s same={same}")
        else:
            print(f"{i:02d} integer={a} dimensions={b} new={dtNew:.6f}s")

    print(f"\npartition total: {newTotal:.6f}s")
    if e == 1:
        print(f"old_partition total: {oldTotal:.6f}s")
        print(f"performance benchmark: {oldTotal / newTotal:.6f}x")


In [ ]:
a = 0 # change to 1 if you'd like to benchmark both the old and new codes
b = 20 # the number of tests to run
c = 16 # the max number of dimensions (partitions) using the formula: 2^c, where if c is 16, then the max number of dimensions will be: 65,536
d = 256 # the max integer value to test using the formula 10^d. If d is 100, then it will test numbers up to 10^100.

timerTest(a, b, c, d)

In [ ]:
a = 1 # if 0 test only new, else if 1 test both
b = 10 # number of tests
c = 16 # the max number of dimensions: 2^c
d = 12 # the max integer value: 10^d

timerTest(a, b, c, d)

In [ ]:
a = 1 # if 0 test only new, else if 1 test both
b = 5 # number of tests
c = 8 # max dimensions
d = 512 # max integer power

timerTest(a, b, c, d)